# CLI rough draft (GPT)

To rewrite the previous program using only MySQL/MariaDB commands without using the `mysql-connector-python` library, we'll use the command-line interface for MySQL. This will require creating and executing SQL scripts and using command-line tools like `mysql` to execute those scripts.

Here's a step-by-step guide to achieve the same functionality using shell scripts and SQL files:

### 1. `setup.sql`

This SQL script will create the required tables in the database.

```sql
CREATE DATABASE IF NOT EXISTS StockDB;
USE StockDB;

CREATE TABLE IF NOT EXISTS `Session` (
    `SessionID` INT PRIMARY KEY
);

CREATE TABLE IF NOT EXISTS `Portfolio` (
    `PortfolioID` INT PRIMARY KEY,
    `TotalAmt` FLOAT,
    `Risk` VARCHAR(64)
);

CREATE TABLE IF NOT EXISTS `Allocation` (
    `AllocID` INT PRIMARY KEY,
    `Ticker` VARCHAR(10),
    `Amount` FLOAT
);

CREATE TABLE IF NOT EXISTS `Stock` (
    `StockID` INT PRIMARY KEY AUTO_INCREMENT,
    `Ticker` VARCHAR(10),
    `Sector` VARCHAR(64),
    `Price` FLOAT,
    `SD` FLOAT,
    `ERet` FLOAT
);

CREATE TABLE IF NOT EXISTS `History` (
    `HistoryID` INT PRIMARY KEY AUTO_INCREMENT,
    `Ticker` VARCHAR(10),
    `Date` DATE,
    `Price` FLOAT
);

CREATE TABLE IF NOT EXISTS `SessionHasPortfolio` (
    `SessionID` INT,
    `PortfolioID` INT,
    FOREIGN KEY (`SessionID`) REFERENCES `Session`(`SessionID`),
    FOREIGN KEY (`PortfolioID`) REFERENCES `Portfolio`(`PortfolioID`),
    PRIMARY KEY (`SessionID`, `PortfolioID`)
);

CREATE TABLE IF NOT EXISTS `PortfolioHasAllocation` (
    `PortfolioID` INT,
    `AllocID` INT,
    FOREIGN KEY (`PortfolioID`) REFERENCES `Portfolio`(`PortfolioID`),
    FOREIGN KEY (`AllocID`) REFERENCES `Allocation`(`AllocID`),
    PRIMARY KEY (`PortfolioID`, `AllocID`)
);

CREATE TABLE IF NOT EXISTS `PortfolioHasStock` (
    `PortfolioID` INT,
    `StockID` INT,
    FOREIGN KEY (`PortfolioID`) REFERENCES `Portfolio`(`PortfolioID`),
    FOREIGN KEY (`StockID`) REFERENCES `Stock`(`StockID`),
    PRIMARY KEY (`PortfolioID`, `StockID`)
);

CREATE TABLE IF NOT EXISTS `AllocationHasStock` (
    `AllocID` INT,
    `StockID` INT,
    FOREIGN KEY (`AllocID`) REFERENCES `Allocation`(`AllocID`),
    FOREIGN KEY (`StockID`) REFERENCES `Stock`(`StockID`),
    PRIMARY KEY (`AllocID`, `StockID`)
);

CREATE TABLE IF NOT EXISTS `StockHasHistory` (
    `StockID` INT,
    `HistoryID` INT,
    FOREIGN KEY (`StockID`) REFERENCES `Stock`(`StockID`),
    FOREIGN KEY (`HistoryID`) REFERENCES `History`(`HistoryID`),
    PRIMARY KEY (`StockID`, `HistoryID`)
);
```

### 2. `fetch_data.py`

This Python script will fetch stock data using `yfinance` and generate SQL insert statements.

```python
import yfinance as yf
from datetime import datetime, timedelta
import sys

def fetch_stock_data(tickers):
    data = {}
    end_date = datetime.now()
    start_date = end_date - timedelta(days=30)
    
    for ticker in tickers:
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        if not stock_data.empty:
            data[ticker] = stock_data
    return data

def generate_sql_inserts(stock_data):
    sql_statements = []
    for ticker, data in stock_data.items():
        last_row = data.iloc[-1]
        sql_statements.append(
            f"INSERT INTO Stock (Ticker, Price) VALUES ('{ticker}', {last_row['Close']});"
        )
        stock_id = f"(SELECT StockID FROM Stock WHERE Ticker='{ticker}')"

        for date, row in data.iterrows():
            sql_statements.append(
                f"INSERT INTO History (Ticker, Date, Price) VALUES ('{ticker}', '{date.strftime('%Y-%m-%d')}', {row['Close']});"
            )
            history_id = f"(SELECT HistoryID FROM History WHERE Ticker='{ticker}' AND Date='{date.strftime('%Y-%m-%d')}')"
            sql_statements.append(
                f"INSERT INTO StockHasHistory (StockID, HistoryID) VALUES ({stock_id}, {history_id});"
            )
    return sql_statements

if __name__ == "__main__":
    tickers = input("Enter the stock tickers separated by commas: ").split(',')
    stock_data = fetch_stock_data(tickers)
    sql_statements = generate_sql_inserts(stock_data)
    
    with open('populate_data.sql', 'w') as f:
        f.write("USE StockDB;\n")
        f.write("\n".join(sql_statements))
```

### 3. `main.sh`

This shell script will run the setup SQL script, fetch stock data, generate the SQL insert statements, and then execute those statements to populate the database.

```bash
#!/bin/bash

# Database credentials
USER="yourusername"
PASSWORD="yourpassword"
DATABASE="StockDB"

# Run setup.sql to create tables
mysql -u $USER -p$PASSWORD < setup.sql

# Fetch stock data and generate populate_data.sql
python3 fetch_data.py

# Populate the database with stock data
mysql -u $USER -p$PASSWORD $DATABASE < populate_data.sql

# Clean up
rm populate_data.sql
```

### Instructions to Run the Program

1. **Install Dependencies:**
   ```bash
   pip install yfinance pandas
   ```

2. **Replace Database Credentials:**
   Replace `yourusername` and `yourpassword` with your actual MySQL credentials in `main.sh`.

3. **Make the Shell Script Executable:**
   ```bash
   chmod +x main.sh
   ```

4. **Run the Shell Script:**
   ```bash
   ./main.sh
   ```

This setup will create the required database and tables, fetch the stock data using `yfinance`, generate the necessary SQL insert statements, and populate the database using the MySQL command-line tool.